## 의미 검색 유사도

### 데이터셋 준비

In [1]:
from datasets import load_dataset

# https://huggingface.co/datasets/klue/klue/viewer/mrc
klue_mrc_dataset = load_dataset('klue', 'mrc', split='train')
klue_mrc_dataset

README.md:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5841 [00:00<?, ? examples/s]

Dataset({
    features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
    num_rows: 17554
})

In [2]:
# 일부 데이터만 사용 row 1000 정도
row_size = 1000
# klue_mrc_dataset[:1000]
klue_mrc_dataset_train = klue_mrc_dataset.train_test_split(train_size=row_size
                                 , shuffle=False)['train']
klue_mrc_dataset_train

Dataset({
    features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
    num_rows: 1000
})

In [3]:
klue_mrc_dataset_train.shape

(1000, 9)

In [4]:
klue_mrc_dataset_train[10].keys()

dict_keys(['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'])

### 데이터 임베딩

In [5]:
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
sentence_model


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [6]:
context_embedded = sentence_model.encode(klue_mrc_dataset_train['context'])
context_embedded.shape

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

(1000, 768)

### 검색 인덱스 생성(KNN 알고리즘 사용)

In [7]:
!pip install faiss-gpu faiss-cpu -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 17.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 44.1 MB/s eta 0:00:00:00:010:01m


In [8]:
import faiss # 메타 API로 벡터 거리 계산용

index_knn = faiss.IndexFlatL2(context_embedded.shape[1])  #  KNN 알고리즘 초기화, 768에 벡터공간 활당
index_knn

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7873be70ea90> >

In [9]:
# 인덱스에 임베딩 저장 : 데이블 생성 유사(메모리용 벡터데이터베이스 유사)
index_knn.add(context_embedded)

In [10]:
type(index_knn), index_knn

(faiss.swigfaiss_avx2.IndexFlatL2,
 <faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7873be70ea90> >)

In [11]:
## 활용
query = '이번 연도에는 언제 비가 많이 올까?'
query_embedded = sentence_model.encode([query])
query_embedded.shape

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(1, 768)

In [12]:
# index_knn.search(query_embedded, 3)
distances, indices = index_knn.search(query_embedded, 3)

distances, indices

(array([[300.91895, 420.8628 , 420.8628 ]], dtype=float32),
 array([[  0, 920, 921]]))

In [13]:
for idx in indices[0]:
    print(klue_mrc_dataset_train['context'][idx][:30])

올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중
연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류
연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류


### 라마 api 벡터 활용

In [14]:
!pip install llama_index llama-index-embeddings-huggingface -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.6/250.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 kB 20.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.


In [15]:
from llama_index.core import VectorStoreIndex, Document

In [16]:
# llama 맞는 dataset 형식 구성
text_list = klue_mrc_dataset[:100]['context']
len(text_list), text_list[3]

(100,
 '미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스터 대학교에서 핵물리학으로 박사 학위를 마쳤다. 제2차 세계대전중에 그는 매사추세츠 공과대학교 방사선 연구소에서 일하였다. 그곳에서 그는 레이다를 개발하였고, 딕 복사계와 마이크로파 수신기를 설계하였다. 그는 방사선 연구소의 지붕에서 이를 20K보다 낮은 우주 마이크로파 배경의 한계 온도를 설정하는 데 사용하였다.\n\n전후 1946년 그는 프린스턴 대학교로 돌아왔고, 이곳에서 자신의 경력의 나머지를 보내게 되었다. 그는 원자 물리학에서 레이저와 전자의 회전 자기 비율을 측정하는 것을 연구하였다. 그의 분광학과 발관 전송 분야의 중대한 기여는 딕 좁아짐(Dicke narrowing, 원자의 자유평균행로가 원자의 방사 전이의 한 파장의 길이보다 짧아지고, 광자의 흡수나 분출하는 동안에 원자의 속도와 방향은 많이 변하는 현상)이라 불리는 현상에 대한 예견이었다. 딕 좁아짐은 단파와 마이크로파 영역에서 상대적으로 낮은 압력에서 발생한다. 딕 좁아짐은 감마선에서 뫼스바우어 효과에 비견된다.\n\n그는 등가 원리를 이용한 일반 상대성 이론의 정확한 측정 프로그램의 개발에 그의 남은 경력을 보냈다. 칼 브랜스(Carl H. Brans)와 함께 그는 중력의 브랜스-딕 이론, 폴 디랙의 큰 수 가설과 마흐의 원리에 의해 영감을 얻은 일반 상대성 이론의 등가 원리의 깨짐을 발전시켰다. 하이라이트는 Roll과 Krotkov와 Dicke에 의한 등가 원리의 고전적인 측정이었고, 이 실험에서 이전의 작업들 보다 100배나 더 정확한 측정치를 얻었다. 디키는 또한 일반 상대성 이론의 고전적 시험 가운데 하나인, 수성의 근일점을 이해하는 데 유용한 태양 편평도를 측정하였다. 폴 디랙은 중력 상수가 우주의 거꾸로 나이와 부정확하게 일치하며, 중력 상수는 지금의 평형을 유지하기 위해 바뀌어야만 한다고 추측했다. 딕은 디랙의 관계가 선택 효과일 수 있다는 것을 깨달았다. 평형이 깨어졌을 

In [17]:
documents = [Document(text=text) for text in text_list] # llama_index에 넣는 형식 맞추기 위해 변형
type(documents), documents[3]

(list,
 Document(id_='0fe97b9b-d9c6-4f5e-a8f0-3cdc27c091ab', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스터 대학교에서 핵물리학으로 박사 학위를 마쳤다. 제2차 세계대전중에 그는 매사추세츠 공과대학교 방사선 연구소에서 일하였다. 그곳에서 그는 레이다를 개발하였고, 딕 복사계와 마이크로파 수신기를 설계하였다. 그는 방사선 연구소의 지붕에서 이를 20K보다 낮은 우주 마이크로파 배경의 한계 온도를 설정하는 데 사용하였다.\n\n전후 1946년 그는 프린스턴 대학교로 돌아왔고, 이곳에서 자신의 경력의 나머지를 보내게 되었다. 그는 원자 물리학에서 레이저와 전자의 회전 자기 비율을 측정하는 것을 연구하였다. 그의 분광학과 발관 전송 분야의 중대한 기여는 딕 좁아짐(Dicke narrowing, 원자의 자유평균행로가 원자의 방사 전이의 한 파장의 길이보다 짧아지고, 광자의 흡수나 분출하는 동안에 원자의 속도와 방향은 많이 변하는 현상)이라 불리는 현상에 대한 예견이었다. 딕 좁아짐은 단파와 마이크로파 영역에서 상대적으로 낮은 압력에서 발생한다. 딕 좁아짐은 감마선에서 뫼스바우어 효과에 비견된다.\n\n그는 등가 원리를 이용한 일반 상대성 이론의 정확한 측정 프로그램의 개발에 그의 남은 경력을 보냈다. 칼 브랜스(Carl H. Brans)와 함께 그는 중력의 브랜스-딕 이론, 폴 디랙의 큰 수 가설과 마흐의 원리에 의해 영감을 얻은 일반 상대성 이론의 등가 원리의 깨짐

In [18]:
# llama index 벡터화
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embedding_model = HuggingFaceEmbedding(model_name='snunlp/KR-SBERT-V40K-klueNLI-augSTS')
embedding_model

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

HuggingFaceEmbedding(model_name='snunlp/KR-SBERT-V40K-klueNLI-augSTS', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7872880b4160>, num_workers=None, max_length=128, normalize=True, query_instruction=None, text_instruction=None, cache_folder=None)

In [19]:
## index 화 
index_llama = VectorStoreIndex.from_documents(documents
                               , embed_model=embedding_model)
index_llama

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
# 활용
query_engine = index_llama.as_query_engine(llm = None)

ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******

In [ ]:
query = '이번 연도에는 언제 비가 많이 올까?'
response = query_engine.query(query)
response